In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
import numpy as np
import sklearn.datasets
import sklearn.cross_validation
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import ensemble
import time
import pickle
import random
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import keras 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.cross_validation import train_test_split
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences


/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
docs = ['Well done!','Good work','Great effort','nice work','Excellent!','Weak','Poor effort!','not good','poor work','Could have done better']
# define class labels
labels = np.array([1,1,1,1,1,0,0,0,0,0])

In [ ]:
#vocab_size = 100
#encoded_docs = [one_hot(d, vocab_size) for d in docs]
#print(encoded_docs)
#train_X[0]

In [ ]:
#Loading the IMDB Training Dataset

In [ ]:
#from keras.datasets import imdb
'''(x_train, y_train),(x_test, y_test) = imdb.load_data(path="imdb.npz",num_words=None,seed=113)from keras.utils.data_utils import get_file
path = get_file('imdb_full.pkl',
               origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')

'''

In [ ]:
#len(x_train[4])
#IMDB DATASET
#f = open('../glove/glove.6B.100d.txt', encoding="utf-8")

print np.shape(train_X)

In [2]:
train_pos=[]
import os
folder_path = '/Users/s0c02nj/Downloads/aclImdb/train/pos'
#it will take the filenames inside the mentioned folder path one at a time
for filename in os.listdir(folder_path):
    #it will open the filename
    f=open(folder_path + '/' + filename)
    #readlines will read the lines of the particular folder
    text=f.readlines()
    train_pos.append(text[0])


In [3]:
#Copying the list of csv's from a folder
train_neg=[]
import os
folder_path_neg = '/Users/s0c02nj/Downloads/aclImdb/train/neg'
for filename in os.listdir(folder_path_neg):
    f=open(folder_path_neg + '/' + filename)
    text1=f.readlines()
    train_neg.append(text1[0])
    
#makes that no of ones as the len of the item given
pos_Y = [1]*len(train_pos)
#makes that no of ones as the len of the item given
neg_Y = [0]*len(train_neg)
#rowwise appends the two arrays
train_X = np.append(train_pos, train_neg)
train_Y = np.append(pos_Y, neg_Y)
#print(len(train_X))
#print(len(train_Y))



In [5]:
print(len(train_Y))

25000


In [6]:
def get_udoc(document):
    utf_doc=[]
    for i in range(0,len(document)):
        #val=unicode(document[i],errors="ignore")
        val=unicode(document[i],'ascii', 'ignore')
        utf_doc.append(val)

    return utf_doc

In [9]:
enc_trainx=get_udoc(train_X)
print enc_trainx[99]

Beautiful film, pure Cassavetes style. Gena Rowland gives a stunning performance of a declining actress, dealing with success, aging, loneliness...and alcoholism. She tries to escape her own subconscious ghosts, embodied by the death spectre of a young girl. Acceptance of oneself, of human condition, though its overall difficulties, is the real purpose of the film. The parallel between the theatrical sequences and the film itself are puzzling: it's like if the stage became a way out for the Heroin. If all american movies could only be that top-quality, dealing with human relations on an adult level, not trying to infantilize and standardize feelings... One of the best dramas ever. 10/10.


In [ ]:
'''enc_trainx=[]
for i in range(0,len(train_X)):
    enc=train_X[i].encode('utf8')
    enc_trainx.append(enc)'''

In [10]:
#Tokenizing the words
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
tokenized_doc=[]
for i in range(0,len(enc_trainx)):
    word=word_tokenize(enc_trainx[i])
    tokenized_doc.append(word)

[nltk_data] Downloading package punkt to /Users/s0c02nj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
print tokenized_doc[0]
#unicode(word, 'ascii', 'ignore')


[u'For', u'a', u'movie', u'that', u'gets', u'no', u'respect', u'there', u'sure', u'are', u'a', u'lot', u'of', u'memorable', u'quotes', u'listed', u'for', u'this', u'gem', u'.', u'Imagine', u'a', u'movie', u'where', u'Joe', u'Piscopo', u'is', u'actually', u'funny', u'!', u'Maureen', u'Stapleton', u'is', u'a', u'scene', u'stealer', u'.', u'The', u'Moroni', u'character', u'is', u'an', u'absolute', u'scream', u'.', u'Watch', u'for', u'Alan', u'``', u'The', u'Skipper', u"''", u'Hale', u'jr.', u'as', u'a', u'police', u'Sgt', u'.']


In [12]:
#Remove Stop Words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stpwrd_free_doc = []
doc=[]

for i in range (len(tokenized_doc)):
    for w in tokenized_doc[i]:
        if w not in stop_words:
            doc.append(w)
    stpwrd_free_doc.append(doc)
    doc=[]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/s0c02nj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
alpha_doc=[]
alpha=[]
for i in range(len(stpwrd_free_doc)):
    for word in stpwrd_free_doc[i]:
        if (word.isalpha()==True):
            alpha.append(word)
    alpha_doc.append(alpha)
    alpha=[]

In [14]:
print alpha_doc[0]

[u'For', u'movie', u'gets', u'respect', u'sure', u'lot', u'memorable', u'quotes', u'listed', u'gem', u'Imagine', u'movie', u'Joe', u'Piscopo', u'actually', u'funny', u'Maureen', u'Stapleton', u'scene', u'stealer', u'The', u'Moroni', u'character', u'absolute', u'scream', u'Watch', u'Alan', u'The', u'Skipper', u'Hale', u'police', u'Sgt']


# Making a dictionary of the words and giving a sequence to the words

In [15]:
#alpha_doc[1]
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#Tokenizing the document and creating am encoding for all the words.
tokenizer = Tokenizer(nb_words=72000)
tokenizer.fit_on_texts(alpha_doc)


/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/keras_preprocessing/text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [ ]:
#Conver


In [16]:
#sequences converts the word to sequences for each doc
sequences = tokenizer.texts_to_sequences(alpha_doc)
#sequences[0]

In [ ]:
#alpha_doc[0]
#len(sequences[2])

In [ ]:
#sequences[0]

# Index of each word as a dictionary

In [17]:
#sequences[0]
#word_index is dictionary of the words and the sequence
word_index = tokenizer.word_index

In [19]:
print word_index['boy']

336


In [20]:
len(word_index)
#type(word_index)
#word_index['the']

71652

In [ ]:
#Checking

In [ ]:
#sequences[11]
len(sequences)
#len(train_X)
len_of_seq=[]
for i in range(0,len(sequences)):
    a=len(sequences[i])
    len_of_seq.append(a)


In [ ]:
#len_of_seq[20149]

# Padding the sequence for input to LSTM

In [21]:
#data stores the padded sequence 
data = pad_sequences(sequences, maxlen=200)

In [22]:
np.shape(data[10000])
len(data)
#LSTM input will take no of time steps

25000

In [ ]:
#data[10000]

# WORD2VEC  Dictionary

In [23]:
folder_path= '/Users/s0c02nj/Downloads/glove.6B/glove.6B.50d.txt'
f=open(folder_path)
doc=f.readlines()
#****VIP
#**WORD TO VEC DICTIONARY
#Forming a dictionary-word2vec
word2vec={}
key=[]
#looping though the doc.in the doc the entire thing is saved and is separated by a space bar.
for line in doc:
    #parts contains every word separately for doc1
    parts=line.split(' ')
    #part[0] contains the word
    word=parts[0]
    key.append(word)
    #embed contains the vector
    embed=np.array(parts[1:],dtype='float32')
    #filling up the dictionary
    word2vec[word]=embed

#NOW IF i give word as the key, the corresponding vec rep will return.

In [25]:
#len(key)
#word2vec['the']

# Embedding Matrix

In [27]:
np.random.seed(0)
x = np.random.random(50)


In [28]:
#word_index['the']>>>>>>>>>>>3
embedding_matrix = np.zeros((len(word_index) + 1, 50))
null_vec=x
embedding_matrix[0]=x
for word, i in word_index.items():
    #print i
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]
    else:
        embedding_matrix[i] = x
 


In [ ]:
#word,i in  word_index.items()[0]
#embedding_matrix
#i

In [31]:
print word_index.items()[4][0]
print np.shape(embedding_matrix)

deferment
(71653, 50)


In [ ]:
#word, i in word_index.items()
#word_index

In [ ]:
#embedding_matrix[0]-x

In [ ]:
embedding_matrix[2].shape
len(word_index)

In [ ]:
embedding_matrix.shape

In [ ]:
#data

In [32]:
#word_index.items()
vocabulary_size=71653
input_length=200

In [33]:
from keras.utils import to_categorical
cat_y=to_categorical(train_Y, num_classes=2)

#x_scaled=sklearn.preprocessing.scale(x, axis=0, with_mean=True, with_std=True, copy=True)
x_train, x_test, y_train, y_test = train_test_split(data, cat_y, test_size=0.2, random_state=42)

In [34]:
print cat_y

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [35]:
np.random.seed(0)
model_glove = Sequential()

In [36]:
model_glove.add(Embedding(vocabulary_size, 50, input_length=200, weights=[embedding_matrix], trainable=True))
#model_glove.add(Dropout(0.2))
#model_glove.add(Conv1D(64, 5, activation='relu'))
#model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100,dropout=0.2))
model_glove.add(Dense(2, activation='softmax'))

In [37]:
model_glove.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model_glove.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 50)           3582650   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 3,643,252
Trainable params: 3,643,252
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#model_glove.summary()

In [39]:
#model_glove.fit(data, np.array(cat_y), validation_split=0.2, epochs=10)